**MehNet**

MehNet: **Me**lting point **H**ashi**n**g **T**able and Vigesimal digits based protein ID numbers

First of all, retrieve needed python libraries.

In [2]:
%pip install Bio
import pandas as pd
from Bio import SeqIO
from collections import defaultdict
import numpy as np
import sys
sys.set_int_max_str_digits(0)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Download datasets from net or local files.

In [3]:
fasta = open("/content/drive/MyDrive/uniprot_sprot.fasta","r")

Parse fasta sequences and remove duplicates. Merge and keep both headers("mkh { }". Exclude fasta sequences which includes aminoacids out of twenty standard ones. Then divide each headers and fastasequences in different lists("seq_ids [ ], fastaseqs [ ]").

In [4]:
dict_fasta = {record.id:record.seq for record in SeqIO.parse(fasta,"fasta")}

mkh = {}
for key, value in dict_fasta.items():
    if value in mkh:
        mkh[value].append(key)
    else:
        mkh[value] = [ key ]




fastaseqs=[]
seq_ids=[]
for seq,id in mkh.items():
    if not "U"in seq:
        if not "X" in seq:
            if not "B" in seq:
                if not "O" in seq:
                    if not "Z" in seq:
                        if not "J" in seq:
                            if not "TERM" in seq:
                                seq_ids.append(id)
                                fastaseqs.append(seq)



Aminoacids ordered via melting points

"EQPKSNCRTDMFIHWGLAVY"

One letter codes assigned to vigesimal digits


Q:0,E:1,C:2,P:3,R:4,K:5,S:6,W:7,N:8,T:9,G:A,D:B,F:C,M:D,I:E,H:F,L:G,A:H,V:I,Y:J


In [5]:
endfastp=[]
for c in fastaseqs:
    r = c.replace("E", "1").replace("Q", "0").replace("C", "2").replace("P", "3").replace("R", "4").replace("K", "5").replace("S", "6").replace("W", "7").replace("N", "8").replace("T", "9").replace("F", "C").replace("H", "F").replace("A", "H").replace("G", "A").replace("D", "B").replace("M", "D").replace("I", "E").replace("L", "G").replace("V", "I").replace("Y", "J")
    endfastp.append(r)

Convert vigesimal to decimal.

If there is an extraordinary aminoacid (different from twenty standard ones; selenocysteine,Pyrrolysine etc.) in sequence, MehNet could not convert and compute it.

Check and count these exceptions. Gives warning("There is at least one kind of aminoacid symbol out of standard list").

In [6]:
ID_number=[]
counter=0
for d in endfastp:
    try:
        decimal= int(str(d),20)
        ID_number.append(decimal)
    except:
        ID_number.append("There is at least one kind of aminoacid symbol out of standard list in=  " + d)
        counter+=1
counter

0

The last step is finding duplicate ID numbers and how many exceptions.

In [7]:
D = defaultdict(list)
for i,item in enumerate(ID_number):
    D[item].append(i)
D = {k:v for k,v in D.items() if len(v)>1}

D


{18807270925240700087000056784566390024958476282094: [436682, 436683]}

Avoid too long integer and literal value error. Convert ID list to array.

In [8]:
ID_number=np.array(ID_number)

Make dataframes and tables of results (IDs)

In [9]:
dataframe=pd.DataFrame()
dataframe["Seq_Id"]=seq_ids
dataframe["fastaps"]=fastaseqs
dataframe["ID_numbers"]=ID_number
dataframe.to_csv("All_IDs.csv")


In [10]:
dataframe.head()

,Seq_Id,fastaps,ID_numbers
0,[sp|Q6GZX4|001R_FRG3G],"(M, A, F, S, A, E, D, V, L, K, E, Y, D, R, R, ...",8036467016895004643795419090889518925170322984...
1,[sp|Q6GZX3|002L_FRG3G],"(M, S, I, I, G, A, T, R, L, Q, N, D, K, S, D, ...",1424363512816199274187851016002172625955902997...
2,[sp|Q197F8|002R_IIV3],"(M, A, S, N, T, V, S, A, Q, G, G, S, N, R, P, ...",5160136090643628425220391833071457704102858576...
3,[sp|Q197F7|003L_IIV3],"(M, Y, Q, A, I, N, P, C, P, Q, S, W, Y, G, S, ...",6372245355580141314328341932374121530513253302...
4,[sp|Q6GZX2|003R_FRG3G],"(M, A, R, P, L, L, G, K, T, S, S, V, R, R, R, ...",4919108260740759386913660578566844213787164069...


Return IDs to original list type

In [11]:
ID_number=ID_number.tolist()

Make dataframes and tables of duplicates

In [12]:
dup=[]
for i in range(len(ID_number)):
    if not i == ID_number.index(ID_number[i]):
        dup.append(i)
        dup.append(ID_number.index(ID_number[i]))



dataframe.iloc[dup]

,Seq_Id,fastaps,ID_numbers
436683,[sp|P0CAL3|VF84L_ASFWA],"(Q, M, M, M, F, I, T, V, Y, D, I, N, Q, K, Q, ...",1880727092524070008700005678456639002495847628...
436682,[sp|P0CAL2|VF84L_ASFP4],"(M, M, M, F, I, T, V, Y, D, I, N, Q, K, Q, K, ...",1880727092524070008700005678456639002495847628...


In [13]:
dataframe.iloc[dup].to_csv("duplicates.csv")